<a href="https://colab.research.google.com/github/iypc-team/CoLab/blob/master/CV2_Utils_Module.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 9/13/2021-4
from __future__ import absolute_import
from IPython.display import clear_output, display
import glob, os, shutil, sys, uuid
from os.path import *
from time import sleep

try:
    from google.colab import drive, files
    from google.colab.patches import cv2_imshow
except Exception as err:
    pass

try: 
    from matplotlib import pyplot as plt
except Exception as err:
    %pip install matplotlib
    from matplotlib import pyplot as plt

try: 
    import cv2
    from cv2 import COLOR_BGR2RGBA
except Exception as err:
    print(err)
    %pip install opencv-python-headless # works.
    # %pip install opencv-contrib-python-headless
    import cv2
    from cv2 import COLOR_BGR2RGBA, IMREAD_UNCHANGED

try:
    import tensorflow as tf
except Exception as err: 
    print(err)
    %pip install tensorflow-gpu
    
import numpy as np

if exists('/content/sample_data'):
    shutil.rmtree('/content/sample_data')

contentPth = os.getcwd()
tensorflowImagesPath = join(contentPth, 'TensorflowImages')
resizedImagesPath = join(contentPth, 'TensorflowResizedImages')

if exists(resizedImagesPath):
    pass
    # shutil.rmtree(resizedImagesPath)

try:
    drive.flush_and_unmount()
    drive.mount('/content/gdrive/', force_remount=True)
    mydrivePth = '/content/gdrive/MyDrive'
    os.chdir(mydrivePth)
    gdriveTFImagesPth = join(mydrivePth, 'TensorflowImages')
    gdriveResizedImagesPth = join(mydrivePth, 'TensorflowResizedImages')
    
    os.chdir(mydrivePth)
    try:
        import TarfileFunctions
        from TarfileFunctions import *
    except Exception as err: print(err)

    from BashColors import C
    copyPath = join(mydrivePth, 'BashColors.py')
    if exists(copyPath):
        shutil.copy2(copyPath, contentPth)
    
    if not exists(tensorflowImagesPath):
        shutil.copytree(gdriveTFImagesPth, tensorflowImagesPath)
        # tff.listTarfiles()
        os.chdir(contentPth)
        # tff.extractTarfiles('Juno_Original_Images.tar.gz')

    gdrive_resized_images = '/content/gdrive/MyDrive/TensorflowResizedImages'
    if not exists(resizedImagesPath):
        try: 
            pass
            # shutil.copytree(gdrive_resized_images, resizedImagesPath)
        except Exception as err:
            print(err)
    os.chdir(contentPth)

except Exception as err:
    # print(err)
    os.chdir(contentPth)
    from TarfileFunctions import *
    from BashColors import C
    tarfilePath=join(contentPth, 'Juno_Original_Images.tar.gz')
    if exists(tarfilePath) and not exists(tensorflowImagesPath):
        tff.extractTarfiles('Juno_Original_Images.tar.gz')

rootPth = expanduser("~")

tensorflowImagesPath = join(contentPth, 'TensorflowImages')
resizedImagesPath = join(contentPth, 'TensorflowResizedImages')

os.chdir(tensorflowImagesPath)
tfImageGlob = glob.glob('**', recursive=True)

os.chdir(contentPth)
cv2ImagesPath = join(contentPth, 'CV2Images')
cv2EnemyPath = join(cv2ImagesPath, 'Enemy')
cv2FriendlyPath = join(cv2ImagesPath, 'Friendly')

if not exists(cv2ImagesPath):
    os.mkdir(cv2ImagesPath)
    os.mkdir(cv2EnemyPath)
    os.mkdir(cv2FriendlyPath)
    
os.chdir(contentPth)
print(f'cv2: {C.IBlue}{cv2.__version__}{C.ColorOff}')
print(f'cwd: {C.IBlue}{os.getcwd()}{C.ColorOff}')
print(f'rootPth: {C.IBlue}{rootPth}{C.ColorOff}')

gpuDevice = tf.test.gpu_device_name()
if gpuDevice != '/device:GPU:0':
    print(f'{C.IRed}Not configured to use a GPU.')
else:
    with tf.device('/device:GPU:0'):
        print(f'Found GPU at: {C.Green}{gpuDevice}')

In [ ]:
with tf.device('/device:GPU:0'):
    pass

In [ ]:

gpuDevice = tf.test.gpu_device_name()
if gpuDevice != '/device:GPU:0':
    print(f'{C.IRed}Not configured to use a GPU.')
else:
    with tf.device('/device:GPU:0'):
        print(f'Found GPU at: {C.Green}{gpuDevice}')

In [ ]:
%%writefile CV2_Utils.py

import cv2
import numpy as np
from matplotlib import pyplot as plt
import os, uuid
from os.path import *
from time import sleep
from BashColors import C

try:
    from google.colab import drive, files
    from google.colab.patches import cv2_imshow
except: pass

try:
    import tensorflow as tf
except Exception as err: print(err)


class CV2_Utils:
    ''' '''
    __all__ = ['cvu','rootPth','contentPth','tfImagesPth','cv2ImagesPth',
               'cv2ImagePathList','tfImagePathList',
               'originalImageZeroPixel', 'zeroPixel']
    # __builtin__ = []

    def __init__(self):
        ''' '''
        self.cvu = CV2_Utils

        self.rootPth = expanduser("~")
        self.contentPth = os.getcwd()
        self.tfImagesPth = join(os.getcwd(), 'TensorflowImages')
        self.cv2ImagesPth = join(os.getcwd(), 'CV2Images')

        self.zeroPixel = np.array([0,0,0,0])
        self.originalImageZeroPixel = np.array([0,0,0,0])

        self.cv2ImagePathList = []
        self.tfImagePathList = []

    def createCV2ImagesTarfile(self):
        ''' '''
        os.chdir(mydrivePth)
        from TarfileFunctions import tff, tarfileFromDirectory
        thisDir = cvu.cv2ImagesPth
        if exists(thisDir):
            # print(thisDir)
            try: tff.tarfileFromDirectory(output_filename='CV2Images.tar.gz',
                                          source_dir=thisDir)
            except BaseException as err: print(err)

    def renameFileWithPath(self, path, append=''):
        '''return newPath'''
        file_path, extension = path.split('.')
        # print(f'file_path: {file_path}')
        # print(f'extension: {extension}')
        newPath = file_path + append + '.' + extension
        newPath = join(newPath)
        
        return newPath
        
    def showTwoImages(self, img1, img2):
        '''display two images with  CV2'''
        img1 = self.resizeImage(img1)
        img2 = self.resizeImage(img2)
        combinedImage = cv2.hconcat([img1, img2])
        try:
            cv2.waitKey(100)
            cv2.destroyAllWindows()
            cv2_imshow(combinedImage)
        except: self.matPlotCompareTwoImages(img1, img2)

    def matPlotShowSingleImage(self, thisImage, title='', showAxis=False):
        ''' '''
        thisImage = cv2.cvtColor(thisImage, cv2.COLOR_BGR2RGBA)

        fig=plt.figure()
        ax1=fig.add_subplot(1,1,1)
        ax1.imshow(thisImage)
        if showAxis: ax1.axis('on')
        else: ax1.axis('off')
        ax1.set_title(title)
        plt.show()

    def matPlotCompareTwoImages(self, thisImage, compareImage,
                                originalTitle='Original Image',
                                newTitle='', showAxis=False):
        ''' '''
        thisImage = cv2.cvtColor(thisImage, cv2.COLOR_BGR2RGBA)
        compareImage = cv2.cvtColor(compareImage, cv2.COLOR_BGR2RGBA)

        fig=plt.figure()
        ax1 = fig.add_subplot(1,2,1)
        ax1.imshow(thisImage)
        if showAxis: ax1.axis('on')
        else: ax1.axis('off')
        ax1.set_title(originalTitle)

        ax2 = fig.add_subplot(1,2,2)
        ax2.imshow(compareImage)
        if showAxis: ax2.axis('on')
        else: ax2.axis('off')
        ax2.set_title(newTitle)
        plt.show()

    def resizeImage(self, thisImage, newSize=(224,224)):
        '''return resized_image'''
        resized_image = cv2.resize(thisImage, newSize,
                                   interpolation=cv2.INTER_CUBIC)
        return resized_image

    def createImageWithColor(self, pxColor, silent=True):
        '''return save_path'''
        bgImage = np.zeros(shape=[224,224,4], dtype=np.uint8)
        save_path = join(self.contentPth, 'bgImage.png')

        for px in bgImage:
            bgImage[:] = pxColor
            sleep(0.1)

        cv2.imwrite(save_path, bgImage)
        
        if not silent:
            print(f'shape: {bgImage.shape}')
            bgImage = cv2.imread(save_path, cv2.IMREAD_UNCHANGED)
            try:
                cv2_imshow(bgImage)
                cv2.waitKey(100)
                cv2.destroyAllWindows()
            except:
                self.matPlotShowSingleImage(bgImage)
        return save_path

    def zoomImage(self, thisImage, newScale=1, silent=True):
        '''return zoomImage'''
        zeroPixel = thisImage[0][0]
        angle = 0
        width, height, _ = thisImage.shape
        rotPoint = width//2, height //2
        dimentions = width, height
        rotPoint = width//2, height//2
        rotMatrix = cv2.getRotationMatrix2D(rotPoint, angle, scale=newScale)
        zoomImage = cv2.warpAffine(thisImage, rotMatrix, dimentions)
        zoomImage = self.cv2FillImage(zoomImage)
        if not silent:
            print('cv2ZoomImage()')
            self.matPlotCompareTwoImages(thisImage, zoomImage)
        return zoomImage

    def rotateImage(self, thisImage,
                    angle=0, rotPoint=None, newScale=1, silent=True):
        '''return rotImage'''
        thisImage = np.copy(thisImage)
        zeroPixel = thisImage[[0][0]]
        width, height, _ = thisImage.shape
        if rotPoint == None:
            rotPoint = width//2, height//2
            rotMat = cv2.getRotationMatrix2D(rotPoint, angle, scale=newScale)
            dimentions = width, height
            rotImage = cv2.warpAffine(thisImage, rotMat, dimentions)
            rotImageZeroPixel = rotImage[0][0]
            if not silent:
                print('cv2Rotation()')
                self.matPlotCompareTwoImages(thisImage, rotImage)
                sleep(0.1)
            try:
                cv2.waitKey(100)
                cv2.destroyAllWindows()
            except: pass
            return rotImage

    def translateImage(self, thisImage, x=0, y=0, silent=True):
        '''-x shift left -y shift up\nx shift right y shift down\n
        return newImage'''
        thisImage = np.copy(thisImage)
        zeroPixel=thisImage[0][0]
        translateMatrix = np.float32([[1,0,x],[0,1,y]])
        dimentions = (thisImage.shape[1], thisImage.shape[0])
        newImage = cv2.warpAffine(thisImage, translateMatrix, dimentions)

        if not silent:
            print('cv2Translate()')
            self.matPlotCompareTwoImages(thisImage, newImage)

        try:
            cv2.waitKey(100)
            cv2.destroyAllWindows()
        except: pass
        return newImage

    def imageEdgeDetection(self, thisImage, t1=50, t2=100, silent=True):
        '''return canny'''

        # gray = cv2.cvtColor(thisImage, cv2.COLOR_BGR2GRAY)
        # thisImage = cv2.GaussianBlur(gray,(3,3),0)

        canny = cv2.Canny(thisImage, t1, t2)
        if not silent:
            self.matPlotCompareTwoImages(thisImage, canny)
        try:
            cv2.waitKey(100)
            cv2.destroyAllWindows()
        except: pass
        return canny

    def changeBackgroundColors(self, path, image=None):
        '''returns newImg'''
        splitPath = split(path)
        filePath = splitPath[0]
        fileName = splitPath[1]
        fileName = 'new_' + fileName
        print(filePath)
        print(fileName)
        saveImagePath = join(contentPth, fileName)

        print(f'saveImagePath: {saveImagePath}')
        original_image = cv2.imread(path, cv2.IMREAD_UNCHANGED)
        originalZeroPixel = original_image[0][0]
        plt.imshow(original_image)
        newImg = np.copy(original_image)

        zeroPixel = newImg[0][0]
        print('newImg zeroPixel:', zeroPixel)
        print('originalZeroPixel:', originalZeroPixel)

        width, height, channels = original_image.shape
        print(width, height, channels)

        for x in range(0, width):
            for y in range(0, height):
                channels_xy = newImg[y][x]
                # print(channels_xy)
                if all(channels_xy == zeroPixel):
                    newImg[y][x] = originalZeroPixel

        cv2.imwrite(saveImagePath, newImg)
        newImg = cv2.imread(saveImagePath, -1)
        try:
            cv2.waitKey(100)
            cv2.destroyAllWindows()
        except: pass
        return newImg

    def cv2FillImage(self, thisImage, silent=True):
        '''returns filledImage'''
        print('cv2FillImage')
        # matPlotShowSingleImage(thisImage)
        zp = thisImage[0][0]
        bgImagePath = self.cv2CreateImageWithColor(pxColor=cvu.zeroPixel)
        # print(zp)
        img1 = np.copy(thisImage)
        img2 = cv2.imread(bgImagePath, cv2.IMREAD_UNCHANGED)
        filledImage = cv2.bitwise_or(img1, img2)
        filledImage = cv2.bitwise_or(filledImage, img2)
        if not silent:
            self.cv2ShowTwoImages(img1, filledImage)
        try:
            cv2.waitKey(100)
            cv2.destroyAllWindows()
        except: pass
        return filledImage

    def addTwoImages(self, imagePath1, imagePath2, alfa=1, beta=1, gamma=0.0):
        '''return  addImage'''
        # read two imagePaths
        src1 = cv2.imread(imagePath1)
        src2 = cv2.imread(imagePath2)
        try:
            # add or blend the imagePaths
            addImage = cv2.addWeighted(src1, alfa, src2, beta, gamma)
            # save the output imagePath
            # cv2.imwrite('image.png', dst)
            return addImage
        except Exception as err:
            print(f'{C.IRed}{err}')

    def flipImage(self, thisImage, axes=0, silent=True):
        '''axes=0 flip vert\naxes=1 flip horiz\naxes=-1 flip vert and horiz'''
        if axes == 0:
            flipImg = cv2.flip(thisImage, 0)
            if not silent:
                print('_flipVert')
                self.showTwoImages(thisImage, flipImg)
        elif axes == 1:
            flipImg = cv2.flip(thisImage, 1)
            if not silent:
                print('_flipHorz')
                self.showTwoImages(thisImage, flipImg)
        elif axes == -1:
            flipImg = cv2.flip(thisImage, -1)
            if not silent:
                print('_flipVertAndHorz')
                self.showTwoImages(thisImage, flipImg)
        try:
            cv2.waitKey(100)
            cv2.destroyAllWindows()
        except: sleep(0.1)
        return flipImg

    def getUniqueFileName(self, uniquePrefix='_'):
        '''return uniqueName'''
        uniqueName = uniquePrefix + str(uuid.uuid4())
        return uniqueName
        
    def getSavePath(self, initialPath, new_path='', postfix='', silent=True):
        ''' '''
        if new_path=='':
            new_path = self.cv2ImagesPth + '/'
        
        directory, fileName = split(initialPath)
        name, extension = splitext(fileName)

        if not silent:
            print(f'{C.IWhite}')
            print(f'newPath: {new_path}')
            print(f'name: {name}')
            print(f'postfix: {postfix}')
            print(f'extension: {extension}')
        
        newSavePath = new_path + name + postfix + extension
        
        print(f'{C.IWhite}newSavePath: {C.IPurple}{newSavePath}{C.ColorOff}')
        return newSavePath

    def saveImage(self, savePath, thisImage, silent=True, saveImage=False):
        ''' '''
        if not silent:
            print(f'{C.IWhite}savePath: {C.Green}{savePath}{C.ColorOff}')
            try: cv2_imshow(thisImage)
            except BaseException as err:
                self.matPlotShowSingleImage(thisImage,
                                            title=basename(savePath))

        if saveImage and not exists(savePath):
            print(f'saved to: {C.Green}{savePath}{C.ColorOff}')
            self.cv2ImagePathList.append(savePath)
            cv2.imwrite(savePath, thisImage)

    def fillImage(thisImage, silent=True):
        ''' '''
        original_image = np.copy(thisImage)
        img = np.copy(original_image)

        black = np.where((img[:,:,0]==0) & 
                         (img[:,:,1]==0) & 
                         (img[:,:,2]==0))
        
        white = np.where((img[:,:,0]==255) & 
                         (img[:,:,1]==255) & 
                         (img[:,:,2]==255))
        
        img[black] = (255, 255, 255, 255)
        img[white] = (0, 0, 0, 255)
        
        if not silent:
            try:
                cvu.showTwoImages(original_image, img)
            except Exception as err:
                print(err)

        try:
            cv2.waitKey(100)
            cv2.destroyAllWindows()
        except: sleep(0.1)

        return img
            
cvu = CV2_Utils()

In [ ]:
def fillImage2(thisImage, silent=True):
    ''' '''
    original_image = np.copy(thisImage)
    img = np.copy(original_image)

    black = np.where((img[:,:,0]==0) & (img[:,:,1]==0) & (img[:,:,2]==0))
    white = np.where((img[:,:,0]==255) & (img[:,:,1]==255) & (img[:,:,2]==255))

    img[black] = (255, 255, 255, 255)
    img[white] = (0, 0, 0, 255)

    if not silent:
        cvu.showTwoImages(original_image, img)


In [ ]:
copyToDrivePath = join(contentPth, 'CV2_Utils.py')
if exists(copyToDrivePath):
    try:
        shutil.copy2(copyToDrivePath, mydrivePth)
        print('ok')
    except: pass

else: pass

In [ ]:
from CV2_Utils import *
print(cvu.__all__, '\n')
# help(cvu)

In [ ]:
from CV2_Utils import *
color = np.array([255, 255, 255, 255])
newColor = np.array([64, 64, 64, 255])

bgImagePath = cvu.createImageWithColor(pxColor=color)
print(bgImagePath)
bgImage = cv2.imread(bgImagePath, cv2.IMREAD_UNCHANGED)
height, width, channels = bgImage.shape

newImg = np.copy(bgImage)
newImg[0:224,0:224] = newColor

pth = join(contentPth, 'bgImage.png')

cvu.matPlotCompareTwoImages(bgImage, newImg,
                            originalTitle=str(color), newTitle=str(newColor))

try:
    cv2.waitKey(100)
    cv2.destroyAllWindows()
except: sleep(0.1)

clear_output()

In [ ]:
tfImagePathList=[]
fullPath=None
os.chdir(tensorflowImagesPath)
tfImageGlob = glob.glob('**', recursive=True)
for pth in sorted(tfImageGlob):
    if pth.endswith('.png'):
        fullPath = abspath(pth)
        tfImagePathList.append(fullPath)
        img = cv2.imread(fullPath, cv2.IMREAD_UNCHANGED)

        try: img = cvu.resizeImage(img)
        except BaseException as err:
            print(err)
            img = cv2.resize(img, (224, 224))
            sleep(0.1)
        
        cvu.saveImage(fullPath, img, silent=False, saveImage=True)

        try:
            cv2.waitKey(100)
            cv2.destroyAllWindows()
        except: sleep(0.1)
        print()
cvu.tfImagePathList = tfImagePathList.copy
clear_output()

In [ ]:
cvu.getSavePath(initialPath=fullPath, silent=False)
help(cvu)

In [ ]:
os.chdir(cvu.cv2ImagesPth)
cv2PathList=[]
cv2GlobList = glob.glob('**', recursive=True)

for fil in sorted(cv2GlobList):
    fullPath = abspath(fil)
    if fullPath.endswith('.png') and not fullPath.endswith('224.png'):
        cv2PathList.append(fullPath)
        print(fullPath)

cvu.cv2ImagePathList = cv2PathList.copy
# clear_output()

In [ ]:
print(len(cv2PathList), 'images\n')
for pth in sorted(cv2PathList):
    print(f'{C.Green}{pth}')
    original = cv2.imread(pth, cv2.IMREAD_UNCHANGED)
    cv2_imshow(original)
    cv2.waitKey(50)
    cv2.destroyAllWindows()
# clear_output()

In [ ]:
for pth in sorted(cv2PathList):
    print(basename(pth))
    original = cv2.imread(pth, cv2.IMREAD_UNCHANGED)
    zeroPixel = original[0][0]

    flip_vert = cvu.flipImage(original, axes=0, silent=True)
    save_path = cvu.getSavePath(pth, postfix='_flipVert')
    cvu.saveImage(save_path, flip_vert, silent=False, saveImage=False)
    print()


    flip_horz = cvu.flipImage(original, axes=1, silent=True)
    save_path = cvu.getSavePath(pth, postfix='_flipHorz')
    print()

    flip_both = cvu.flipImage(original, axes=-1, silent=True)
    save_path = cvu.getSavePath(pth, postfix='_flipVertAndHorz')
    print()

    cv2.waitKey(50)
    cv2.destroyAllWindows()
    print()

In [ ]:

os.chdir(cvu.cv2ImagesPth)
cv2Glob = glob.glob('**', recursive=True)

for fil in sorted(cv2Glob):
    if fil.endswith('.png'):
        fullPath=abspath(fil)

        # savePath = cvu.getSavePath(fullPath, silent=False)
        savePath = cvu.renameFileWithPath(fullPath, append='Zoom50')
        # print(savePath)

        print(basename(fullPath))
        img = cv2.imread(fullPath, cv2.IMREAD_UNCHANGED)
        rotImg = cvu.rotateImage(img, newScale=0.5)
        name = basename(savePath)
        cvu.matPlotCompareTwoImages(img, rotImg, title=name)

        imgEdges = cvu.imageEdgeDetection(img)
        rotImgEdges = cvu.imageEdgeDetection(rotImg)
        cvu.matPlotCompareTwoImages(imgEdges, rotImgEdges, title=name)

        cv2.waitKey(250)
        cv2.destroyAllWindows()

os.chdir(cvu.contentPth)
# clear_output()

In [ ]:

cvu.originalImageZeroPixel = img[0][0]
cvu.zeroPixel=img[0][0]
print(cvu.zeroPixel)
rotImg = cvu.rotateImage(img, newScale=0.75, silent=False)

for px in rotImg:
    if px.all() != cvu.originalImageZeroPixel.all():
        print(f'{px}')
        
        pass
    # print(f'{C.IBlue}-—-—-—-—-—{C.IWhite}')

# clear_output()

In [ ]:
test_img = cvu.flipImage(img, axes=-1)
test_img = cvu.rotateImage(test_img,newScale=0.75)

try: cv2_imshow(test_img)
except: cvu.matPlotShowSingleImage(test_img)

In [ ]:

# Read the image
original_image = np.copy(test_img)

zp = original_image[0][0]
zeroPixelTuple = (zp[0],zp[1],zp[2], 255)
print('zeroPixelTuple:', zeroPixelTuple)
# Not necessary. Make a copy to plot later
img = np.copy(original_image)

#Isolate the areas where the color is black(every channel=0) and white (every channel=255)
black = np.where((img[:,:,0]==0) & (img[:,:,1]==0) & (img[:,:,2]==0))

gray = np.where((img[:,:,0]==64) & (img[:,:,1]==64) & (img[:,:,2]==64))

white = np.where((img[:,:,0]==255) & (img[:,:,1]==255) & (img[:,:,2]==255))

#Turn black pixels to white and vice versa
img[black] = (255, 255, 255, 255)
img[gray] = (64, 64, 64, 255)
img[white] = (0, 0, 0, 255)
print(img[black].all())

# Plot the images
# cvu.showTwoImages(original_image,img)
cvu.matPlotCompareTwoImages(original_image, img, newTitle='Filled Image')

In [ ]:

print(zp)
for val in zp:
    print(val)

print(zeroPixelTuple)
print(zeroPixelTuple[0])
print(zeroPixelTuple[1])
print(zeroPixelTuple[2])
print(zeroPixelTuple[3])

In [ ]:
for pth in sorted(tfImagePathList):
    print(basename(pth))
    img = cv2.imread(pth)
    img = cvu.resizeImage(img)
    try: 
        img = cvu.fillImage(img)
        cv2_imshow(img)
    except: pass
    